In [63]:
#Importing Libraries
import pandas as pd
import smtplib
import io, os
import base64
import requests
import time
import boto3, json
from datetime import datetime, timedelta, date
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from PIL import Image, ImageDraw, ImageFont, ImageOps

In [64]:
#Sender details
sender = "contact@tarezameenfoundation.org"
password = "tzfhr@2021"

creds_link = r"C:\Users\jaske\OneDrive\Desktop\NGO\credentials.json"
birthday_email_list = r"C:\Users\jaske\OneDrive\Desktop\NGO\email\birthday2022\Birthday_Email_List.xlsx"
html_file = r"C:\Users\jaske\OneDrive\Desktop\NGO\email\birthday2022\html2022.html"
raw_image = r"C:\Users\jaske\OneDrive\Desktop\NGO\email\birthday2022\body.jpg"
modified_image = r"C:\Users\jaske\OneDrive\Desktop\NGO\email\birthday2022\body_modified.jpg"
text_font = r"C:\Users\jaske\OneDrive\Desktop\NGO\email\birthday2022\ArialCEBold.ttf"
old_error_output = r"C:\Users\jaske\OneDrive\Desktop\NGO\email\birthday2022\error_folder\error_output_{}.txt".format( (datetime.now() - timedelta(days = 1)).strftime("%d-%m-%Y") )
new_error_output = r"C:\Users\jaske\OneDrive\Desktop\NGO\email\birthday2022\error_folder\error_output_{}.txt".format( datetime.now().strftime("%d-%m-%Y") )

In [65]:
#Configuring AWS
creds = open(creds_link)
data = json.load(creds)
creds.close()

region_name = "ap-south-1"
boto3_session = boto3.Session(
aws_access_key_id = data["aws"]["access_key_id"],
aws_secret_access_key = data["aws"]["secret_access_key"],
region_name = region_name)

s3 = boto3_session.resource('s3')
processed_path = "s3://tarezameenfoundation/Birthday/processed/"
bucket = processed_path.split("/")[2]

In [66]:
#Remove older error file
try:
    os.remove(old_error_output)
except Exception as e:
    pass

In [67]:
#Getting clean Data
today = (datetime.now()-timedelta(days = 0)).strftime("%d-%m")
df = pd.read_excel(birthday_email_list)

df['Name'] = df['Name'].str.strip()
df['Name'] = df['Name'].str.lower()
df['Email'] = df['Email'].str.strip()
df['Email'] = df['Email'].str.lower()

df.dropna(subset=["date of Birth"], inplace=True)
df["date of Birth"]= pd.to_datetime(df["date of Birth"], format='%Y-%m-%d')
df = df[df["date of Birth"].dt.strftime("%d-%m") == today]
df = df.drop_duplicates(subset = ["Email"])
df = df.reset_index(drop=True)
df

,Name,Email,date of Birth
0,papiya paul,papiyapaul2405@gmail.com,2001-05-24
1,agniva bandyopadhyay,banerjeeagniva4999@gmail.com,2001-05-24
2,vivek sharma,vivekpjr431@gmail.com,2001-05-24
3,vandhana kumari,vkvermamay1990@gmail.com,1990-05-24
4,naunihal yadav,naunihal9920@gmail.com,2002-05-24
5,sanya mahajan,sanyamaha1124@gmail.com,2002-05-24
6,ravish javed,ravishjaved6@gmail.com,1999-05-24
7,sivaranjani. n,www.ranjani2428@gmail.com,2002-05-24
8,ramyashree s,20d4410@humanities.claretcollege.edu.in,2002-05-24
9,rohini bathula,rohinipatel2427@gmail.com,2002-05-24


In [62]:
#Main message
for i in df.index:
    message = MIMEMultipart()
    message["From"] = sender
    message["Subject"] = f"Hey {df['Name'][i].title()}, It's your birthday!! ..."
    message["To"] = df["Email"][i]
    
    file = open(html_file, "r")
    body = file.read()
    file.close()
    
    #Changing Body
    img = Image.open(raw_image)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(text_font, size=75)
    
    name = df["Name"][i].split(" ")
    if(len(name) > 1):
        name = " ".join([name[0], name[-1]]).title()
    else:
        name = name[0].title()
    
    color = "rgb(255,255,255)"
    w, h = draw.textsize(name, font=font)
    draw.text(((1100-w)/2, 560), name, fill=color, font=font)
    img.save(modified_image)
    
    #Putting file into s3
    modified_image_key = "/".join(processed_path.split("/")[3:])+"{}.jpg".format(name.replace(" ", "_"))
    s3.Bucket(bucket).upload_file(modified_image, modified_image_key, ExtraArgs={'ACL': 'bucket-owner-full-control'})
    s3_url = f"https://{bucket}.s3.{region_name}.amazonaws.com/{modified_image_key}"
    try: 
        time.sleep(10)
        message.attach(MIMEText(body.format(s3_url),"html"))

        session = smtplib.SMTP("smtp.gmail.com", 587)
        session.starttls()
        session.login(sender, password)
        text = message.as_string()
        session.sendmail(sender, df["Email"][i], text)
        session.quit()
    except Exception as e:
        append_mode = "a" if os.path.exists(new_error_output) else "w"
        f = open(new_error_output, append_mode)
        f.write(str(e))
        f.close()
        pass
    
    del message["From"]
    del message["Subject"]
    del message["To"]